Packages:

In [57]:
import glob
import os
import warnings
 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pydicom
from pydicom.data import get_testdata_files
from sklearn.model_selection import train_test_split

from IPython.display import Markdown, display

 # Import functions from the module
import importlib
import help_files._0_definitions 
# import  help_files._1_visuals_script
# import  help_files._01_load_data
 # Reload the module to apply the changes to the script
importlib.reload(help_files._0_definitions)
 
# importlib.reload(help_files._01_load_data)
 
# Group by 'condition', 'level', and 'severity' and count occurrences
from help_files._0_definitions import count_severity_by_condition_level 
# Define the path
from pathlib import Path

pd.set_option("display.width", 1000)  # Set a large width to prevent line wrapping
 

In [58]:
### In definitions are all the functions that are used in the notebook and globals
with open("help_files/_0_definitions.py") as file:
    exec(file.read())
    ### In definitions are all the functions that are used in the notebook and globals
with open("help_files/_0_run_definitions.py") as file:
    exec(file.read())

In [59]:
# loading data
file_names = ["train_df_3_cat.csv", "test_df_3_cat.csv"]
# Load the data from the CSV files
dataframes = [pd.read_csv(data_path_vor / file_name) for file_name in file_names]
# Unpack the dataframes into separate variables
train_df, test_df = dataframes

print("DataFrames have been loaded successfully.")



DataFrames have been loaded successfully.


In [60]:
train_df.dtypes

study_id           int64
severity         float64
condition          int64
level             object
series_id        float64
x                float64
y                float64
image_path        object
missing_image       bool
dtype: object

In [61]:
# # end sample or small sample    
if whole_data_set == False:
    print("Using the whole data set")
else:
    train_df = train_df.sample(n=100, random_state=RSEED)
    test_df = test_df.sample(n=100, random_state=RSEED)
    display(Markdown('<span style="color:red"> this is a small sample : 48692</span>'))

<span style="color:red"> this is a small sample : 48692</span>

Ttrasformaiton and dataloader (do not change over models)

In [62]:
# define transformation and class

import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader, random_split
# %pip install torch torchvision
import torchvision.models as models
from torchvision import transforms
import pydicom
# %pip install opencv-python
import cv2
import pandas as pd


# Define the transform with augmentation: I already tranformed i tbfore 

transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    #transforms.RandomHorizontalFlip(),  # Randomly flip the image horizontally
    #transforms.RandomRotation(2),       # Randomly rotate the image by ±10 degrees
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Adjust color properties
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


class MRIDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

        # Ensure severity is in integer format
        self.data['severity'] = self.data['severity'].astype(int)

    def __getitem__(self, index):
        row = self.data.iloc[index]
        image_path = row['image_path']
        label = row['severity']  # Use severity for the label

        dicom_image = pydicom.dcmread(image_path)
        image = dicom_image.pixel_array.astype(float)
        image = (image / image.max() * 255).astype('uint8')  # Normalize

        # Convert the image to RGB if it is grayscale
        if len(image.shape) == 2:  # Grayscale
            image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)

        # Apply transformations including augmentation
        image_tensor = self.transform(image) if self.transform else torch.from_numpy(image).permute(2, 0, 1)

        return image_tensor, torch.tensor(label).long()  # Return label as tensor

    def __len__(self):
        return len(self.data)



In [63]:
train_data = train_df

training model resnet50 sotp if validation and train go apart

* Setting the model

In [64]:
import mlflow
import mlflow.pytorch
import random
# Set random seed for reproducibility
seed = 42  # You can choose any integer
torch.manual_seed(seed)
np.random.seed(seed)  # Set seed for numpy
random.seed(seed)  # Set seed for random module

# Create a new experiment
Resnet50_class_seg1_seg1_basic = "Resnet50_class_seg1_seg1_basic"
mlflow.set_experiment(Resnet50_class_seg1_seg1_basic)

<Experiment: artifact_location='file:///c:/Users/HP1/Desktop/Spiced/capstone-project/mlruns/711328181724227740', creation_time=1731147338980, experiment_id='711328181724227740', last_update_time=1731147338980, lifecycle_stage='active', name='Resnet50_class_seg1_seg1_basic', tags={}>

In [65]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import models
# %pip install keras
# %pip install tensorflow
# from tensorflow.keras.callbacks import EarlyStopping  # Import EarlyStopping
import numpy as np  # Import numpy for setting the random seed

# Create the dataset
dataset = MRIDataset(data=train_data, transform=transform)

# Split the dataset into training and validation sets (80% train, 20% validation)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

batch_size = 256
# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Load ResNet-50 and set up for classification
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_classes = train_df['severity'].nunique()

model = models.resnet50(weights='IMAGENET1K_V1')
model.fc = nn.Linear(model.fc.in_features, num_classes)
model.to(device)

# Define loss functions and optimizer
learining_rate = 0.0001 
criterion_cel = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learining_rate)
num_epochs = 20

# Early stopping parameters
# Lists to store loss values for plotting
train_losses_cel = []
val_losses_cel = []

# Early stopping parameters
diverge_count = 0              # Counter for divergence-based stopping
stop_threshold = 0.3           # Threshold for divergence
max_diverge_count = 3          # Max number of epochs with diverging validation loss
patience = 5  # Number of epochs to wait for improvement before early stopping


# Calculate number of layers in the model
num_layers = len(list(model.children()))


* setting mlflow: end running new model

In [66]:
import torch
import numpy as np
import os
import mlflow
import matplotlib.pyplot as plt

# Start MLflow run
with mlflow.start_run():
    # Log parameters
    mlflow.log_param("learning_rate", 'learning_rate')
    mlflow.log_param("optimizer", "Adam")
    mlflow.log_param("batch_size", 'batch_size')
    mlflow.log_param("num_epochs", num_epochs)
    mlflow.log_param("num_classes", num_classes)
    mlflow.log_param("model_architecture", "ResNet-50")
    
    # Log early stopping parameters
    mlflow.log_param("stop_threshold", stop_threshold)
    mlflow.log_param("max_diverge_count", max_diverge_count)
    mlflow.log_param("patience", patience)

    # Set descriptive tags for the model
    mlflow.set_tag("model_description", "ResNet-50 for 3 cat and Sagittal T2/STIR and Sagittal T1 images")

    # Example input tensor with the same shape as the model's expected input
    example_input = torch.randn(1, 3, 224, 224)  # Batch size 1, 3 color channels, 224x224 image
    best_val_loss = float('inf')  # Initialize best validation loss
    patience_counter = 0  # Plateau counter
    
    diverge_count = 0  # Counter for divergence-based stopping

    train_losses_cel = []  # List to store training losses
    val_losses_cel = []  # List to store validation losses

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        running_loss_cel_train = 0.0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            
            outputs = model(images)
            
            # Cross Entropy Loss
            loss_cel = criterion_cel(outputs, labels)
            running_loss_cel_train += loss_cel.item()
            
            # Backward pass and optimization
            loss_cel.backward()
            optimizer.step()

        # Calculate average training loss for the epoch
        epoch_loss_cel_train = running_loss_cel_train / len(train_loader)
        train_losses_cel.append(epoch_loss_cel_train)

        # Log training loss to MLflow
        mlflow.log_metric("train_loss_cel", epoch_loss_cel_train, step=epoch)

        # Validation phase
        model.eval()
        running_loss_cel_val = 0.0

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                
                # Cross Entropy Loss for validation
                loss_cel = criterion_cel(outputs, labels)
                running_loss_cel_val += loss_cel.item()

        # Calculate validation loss for the epoch
        epoch_loss_cel_val = running_loss_cel_val / len(val_loader)
        val_losses_cel.append(epoch_loss_cel_val)

        # Log validation loss to MLflow
        mlflow.log_metric("val_loss_cel", epoch_loss_cel_val, step=epoch)

        print(f'Epoch [{epoch + 1}/{num_epochs}], '
              f'Train Cross Entropy Loss: {epoch_loss_cel_train:.4f}, '
              f'Validation Cross Entropy Loss: {epoch_loss_cel_val:.4f}')

        # Save weights for each epoch
        epoch_weight_path = f"model_weights_epoch_{epoch + 1}.pt"
        torch.save(model.state_dict(), epoch_weight_path)
        mlflow.log_artifact(epoch_weight_path)  # Log model weights as artifact for each epoch
        os.remove(epoch_weight_path)  # Optionally, delete the local file after logging

        # Check for early stopping (divergence-based stopping)
        if epoch_loss_cel_val > epoch_loss_cel_train * (1 + stop_threshold):
            diverge_count += 1
            if diverge_count >= max_diverge_count:
                print(f"Early stopping at epoch {epoch + 1} due to validation loss diverging.")
                break
        else:
            diverge_count = 0  # Reset diverge count if validation loss is not diverging

        # Check for early stopping (plateau-based stopping)
        if epoch_loss_cel_val < best_val_loss:
            best_val_loss = epoch_loss_cel_val
            patience_counter = 0  # Reset plateau counter if validation loss improves
            # Save the model weights when validation loss improves (best model)
            best_model_weight_path = f"best_model_weights_epoch_{epoch + 1}.pt"
            torch.save(model.state_dict(), best_model_weight_path)
            mlflow.log_artifact(best_model_weight_path)  # Log the best model weights
            os.remove(best_model_weight_path)  # Optionally, delete the local file after logging
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping at epoch {epoch + 1} due to lack of validation loss improvement.")
                break

    print("Training complete!")

    # Log final model
    example_input_np = example_input.numpy()
    mlflow.pytorch.log_model(model, "final_model", input_example=example_input_np)

    # Plot and log the loss curves as artifacts
    plt.figure(figsize=(10, 5))
    plt.plot(train_losses_cel, label='Train Cross Entropy Loss')
    plt.plot(val_losses_cel, label='Validation Cross Entropy Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training and Validation Cross Entropy Loss')
    plt.legend()
    plt.savefig("cross_entropy_loss.png")
    mlflow.log_artifact("cross_entropy_loss.png")


FileNotFoundError: [Errno 2] No such file or directory: 'data/train_images_origin/2383836357/217610363/16.dcm'

In [ ]:
ss

In [ ]:
loaded_model = mlflow.pytorch.load_model("runs:/deaa903e488043d19aad572a6c429479/final_model")
print(loaded_model)

In [ ]:
import torch
import torchvision.models as models

# Step 1: Define the model architecture (same as training)
model_for_prediciton = models.resnet50()
model_for_prediciton.fc = torch.nn.Linear(model_for_prediciton.fc.in_features, 3)  # Update for your classes

# Step 2: Load the best model weights
# Adjust this path if necessary
best_weights_path = "C:/Users/HP1/Desktop/Spiced/capstone-project/mlruns/711328181724227740/deaa903e488043d19aad572a6c429479/artifacts/best_model_weights_epoch_1.pt"
model_for_prediciton.load_state_dict(torch.load(best_weights_path))

# Step 3: Set the model to evaluation mode
model_for_prediciton.eval()
model_for_prediciton.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))

In [15]:
# Replace <run_id> with your actual run ID and <path_to_model> with the artifact path used during logging
model_path = r"C:\Users\HP1\Desktop\Spiced\capstone-project\mlruns\711328181724227740\deaa903e488043d19aad572a6c429479\artifacts\final_model"
model_for_prediciton = mlflow.pytorch.load_model(model_path)

model_for_prediciton.eval()  # Set the model to evaluation mode

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

Restore parameters and metrtics from mlflow

I should take prdicted probabilities and not predicted classes: output scores (logits) are converted into probabilities using the softmax function

In [16]:
# class definition
import mlflow
import mlflow.pytorch
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision.models as models
from torchvision import transforms
import pydicom
import cv2
import pandas as pd
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np



# Load the model
model = model_for_prediciton
model.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))  # Move model to appropriate device
model.eval()  # Set the model to evaluation mode

# Prepare data
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

class MRIDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform
        self.data['severity'] = self.data['severity'].astype(int)

    def __getitem__(self, index):
        row = self.data.iloc[index]
        image_path = row['image_path']
        label = row['severity']
        dicom_image = pydicom.dcmread(image_path)
        image = dicom_image.pixel_array.astype(float)
        image = (image / image.max() * 255).astype('uint8')
        if len(image.shape) == 2:  # Grayscale
            image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
        image_tensor = self.transform(image) if self.transform else torch.from_numpy(image).permute(2, 0, 1)
        return image_tensor, torch.tensor(label).long()

    def __len__(self):
        return len(self.data)

predicting probabilities for test data

In [17]:
test_data = test_df

In [18]:
# calculate prbabilities and probability list
test_dataset = MRIDataset(data=test_data, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Inference loop with probability extraction
results = []
probabilities_list = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)  # Ensure labels are also on the correct device
        outputs = model(images)
        probabilities = torch.softmax(outputs, dim=1)  # Calculate class probabilities
        _, predicted_classes = torch.max(outputs, 1)
        
        # Append predictions and probabilities
        results.append(predicted_classes.item())
        probabilities_list.append(probabilities.cpu().numpy())



Using device: cpu


In [19]:
# Print predicted classes and their probabilities
for i, (pred, probs) in enumerate(zip(results, probabilities_list)):
    print(f"Test image {i}: Predicted class {pred}, Probabilities: {probs}")

# Additional code to plot the confusion matrix can stay as-is:
true_labels = []
predicted_labels = []

for images, labels in test_loader:
    images = images.to(device)
    true_labels.extend(labels.numpy())

    with torch.no_grad():
        outputs = model(images)
        _, predicted_classes = torch.max(outputs, 1)
        predicted_labels.extend(predicted_classes.cpu().numpy())

true_labels = np.array(true_labels)
predicted_labels = np.array(predicted_labels)

print("True Labels Unique Values:", np.unique(true_labels))
print("Predicted Labels Unique Values:", np.unique(predicted_labels))


Test image 0: Predicted class 0, Probabilities: [[9.9618053e-01 3.8086153e-03 1.0922757e-05]]
Test image 1: Predicted class 1, Probabilities: [[1.4524867e-08 9.9999774e-01 2.2434274e-06]]
Test image 2: Predicted class 1, Probabilities: [[0.00616368 0.99165326 0.00218306]]
Test image 3: Predicted class 1, Probabilities: [[2.4807669e-05 9.9992085e-01 5.4300079e-05]]
Test image 4: Predicted class 1, Probabilities: [[6.2356039e-04 9.9937123e-01 5.2000514e-06]]
Test image 5: Predicted class 1, Probabilities: [[2.5383797e-03 9.9739408e-01 6.7593719e-05]]
Test image 6: Predicted class 1, Probabilities: [[0.01508838 0.805109   0.1798026 ]]
Test image 7: Predicted class 1, Probabilities: [[2.2381102e-05 9.9987686e-01 1.0066373e-04]]
Test image 8: Predicted class 1, Probabilities: [[6.9637747e-07 9.9986064e-01 1.3865811e-04]]
Test image 9: Predicted class 0, Probabilities: [[0.9500599  0.04504139 0.00489869]]
Test image 10: Predicted class 0, Probabilities: [[9.9889874e-01 1.0764807e-03 2.471018

KeyboardInterrupt: 

In [20]:
# Convert the list of probabilities to a numpy array for easier manipulation
probabilities_array = np.vstack(probabilities_list)

# Ensure the length of probabilities_array matches the length of test_df
probabilities_array = probabilities_array[:len(test_df)]

# Add the probabilities to the test_df DataFrame
test_df['Probability_Class_0'] = probabilities_array[:, 0]
test_df['Probability_Class_1'] = probabilities_array[:, 1]
test_df['Probability_Class_2'] = probabilities_array[:, 2]

# Round the probabilities to 4 decimal places
test_df['Probability_Class_0'] = test_df['Probability_Class_0'].round(4)
test_df['Probability_Class_1'] = test_df['Probability_Class_1'].round(4)
test_df['Probability_Class_2'] = test_df['Probability_Class_2'].round(4)
# Add the predicted classes to the test_df DataFrame
test_df['Predicted_Class'] = results



# Display the updated DataFrame
 
test_df.head()

,study_id,severity,condition,level,series_id,x,y,image_path,missing_image,Probability_Class_0,Probability_Class_1,Probability_Class_2,Predicted_Class
0,4003253,0,0,l5/s1,1.054714e+09,197.100569,289.457306,data/train_images_origin/4003253/702807833/3.dcm,False,0.9962,0.0038,0.0000,0
1,4646740,1,0,l5/s1,3.486248e+09,235.980844,360.313610,data/train_images_origin/4646740/3486248476/2.dcm,False,0.0000,1.0000,0.0000,1
2,4646740,1,0,l5/s1,3.486248e+09,235.980844,360.313610,data/train_images_origin/4646740/3666319702/16...,False,0.0062,0.9917,0.0022,1
3,4646740,1,0,l5/s1,3.486248e+09,235.980844,360.313610,data/train_images_origin/4646740/3486248476/16...,False,0.0000,0.9999,0.0001,1
4,4646740,1,0,l5/s1,3.486248e+09,235.980844,360.313610,data/train_images_origin/4646740/3666319702/10...,False,0.0006,0.9994,0.0000,1


In [21]:
# Group by severity and sum the Probability_Class_1
severity_prob_sum_all_classes = test_df.groupby('severity')[['Probability_Class_0', 'Probability_Class_1', 'Probability_Class_2']].mean()

# Print the result
severity_prob_sum_all_classes
# Use severity_prob_sum_all_classes as confusion matrix
conf_matrix_prob_df = severity_prob_sum_all_classes 
# Create a confusion matrix from severity and predicted class
conf_matrix_severity_pred = confusion_matrix(test_df['severity'], test_df['Predicted_Class'])

# Create a DataFrame from the confusion matrix
conf_matrix_severity_pred_df = pd.DataFrame(
    conf_matrix_severity_pred, 
    index=[f"Actual {i}" for i in range(len(conf_matrix_severity_pred))], 
    columns=[f"Predicted {i}" for i in range(len(conf_matrix_severity_pred[0]))]
)

# Print the confusion matrix DataFrame
print(conf_matrix_severity_pred_df)

# Optionally, display it using a more formatted view (e.g., in Jupyter Notebook)
conf_matrix_severity_pred_df.style.background_gradient(cmap='Blues')
# Print the confusion matrix DataFrame
print(conf_matrix_prob_df)

# Optionally, display it using a more formatted view (e.g., in Jupyter Notebook)
conf_matrix_prob_df.style.background_gradient(cmap='Blues')


          Predicted 0  Predicted 1  Predicted 2
Actual 0         2705          386           31
Actual 1           84         3191           31
Actual 2           35          173         2937
          Probability_Class_0  Probability_Class_1  Probability_Class_2
severity                                                               
0                    0.844914             0.143171             0.011912
1                    0.031295             0.957033             0.011667
2                    0.020081             0.069996             0.909923


,Probability_Class_0,Probability_Class_1,Probability_Class_2
severity,,,
0,0.844914,0.143171,0.011912
1,0.031295,0.957033,0.011667
2,0.020081,0.069996,0.909923


In [ ]:
conf_matrix_severity_pred_df.style.background_gradient(cmap='Blues')

In [ ]:
from sklearn.metrics import confusion_matrix
import pandas as pd
import numpy as np

# Ensure true_labels and predicted_labels have the same length
min_length = min(len(true_labels), len(predicted_labels))
true_labels = true_labels[:min_length]
predicted_labels = predicted_labels[:min_length]

# Generate the confusion matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)

# Create a DataFrame from the confusion matrix
conf_matrix_df = pd.DataFrame(
    conf_matrix, 
    index=[f"Actual {i}" for i in range(len(conf_matrix))], 
    columns=[f"Predicted {i}" for i in range(len(conf_matrix[0]))]
)

# Print the confusion matrix DataFrame
print(conf_matrix_df)

# Optionally, display it using a more formatted view (e.g., in Jupyter Notebook)
conf_matrix_df.style.background_gradient(cmap='Blues')


In [22]:
from sklearn.metrics import precision_score, recall_score, accuracy_score
# Calculate precision, recall, and accuracy using mean_probabilities_df
precision = precision_score(true_labels, predicted_labels, average='weighted')
recall = recall_score(true_labels, predicted_labels, average='weighted')
accuracy = accuracy_score(true_labels, predicted_labels)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Accuracy: {accuracy:.4f}")


Precision: 0.8903
Recall: 0.8788
Accuracy: 0.8788
